In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [3]:
df=pd.read_csv("/workspace/data/miml_dataset/miml_labels_1.csv")
columns=["desert", "mountains", "sea", "sunset", "trees"]
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
dataframe=df[:1800],
directory="/workspace/data/miml_dataset/images",
x_col="Filenames",
y_col=columns,
batch_size=32,
seed=42,
shuffle=True,
class_mode="other",
target_size=(100,100))
valid_generator=test_datagen.flow_from_dataframe(
dataframe=df[1800:1900],
directory="/workspace/data/miml_dataset/images",
x_col="Filenames",
y_col=columns,
batch_size=32,
seed=42,
shuffle=True,
class_mode="other",
target_size=(100,100))
test_generator=test_datagen.flow_from_dataframe(
dataframe=df[1900:],
directory="/workspace/data/miml_dataset/images",
x_col="Filenames",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(100,100))

Found 1800 validated image filenames.
Found 100 validated image filenames.
Found 100 validated image filenames.


In [4]:
df = pd.read_csv("/workspace/data/miml_dataset/miml_labels_2.csv")
df["labels"]=df["labels"].apply(lambda x:x.split(","))

In [5]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
dataframe=df[:1800],
directory="/workspace/data/miml_dataset/images",
x_col="Filenames",
y_col="labels",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["desert", "mountains", "sea", "sunset", "trees"],
target_size=(100,100))
valid_generator=test_datagen.flow_from_dataframe(
dataframe=df[1800:1900],
directory="/workspace/data/miml_dataset/images",
x_col="Filenames",
y_col="labels",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["desert", "mountains", "sea", "sunset", "trees"],
target_size=(100,100))
test_generator=test_datagen.flow_from_dataframe(
dataframe=df[1900:],
directory="/workspace/data/miml_dataset/images",
x_col="Filenames",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(100,100))

Found 1800 validated image filenames belonging to 5 classes.
Found 100 validated image filenames belonging to 5 classes.
Found 100 validated image filenames.


In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
56/56 [==============================] - 49s 874ms/step - loss: 0.5009 - acc: 0.7685 - val_loss: 0.5512 - val_acc: 0.7104
Epoch 2/10
56/56 [==============================] - 35s 624ms/step - loss: 0.4277 - acc: 0.8097 - val_loss: 0.5461 - val_acc: 0.7441
Epoch 3/10
56/56 [==============================] - 35s 618ms/step - loss: 0.3952 - acc: 0.8248 - val_loss: 0.4233 - val_acc: 0.8382
Epoch 4/10
56/56 [==============================] - 50s 893ms/step - loss: 0.3765 - acc: 0.8328 - val_loss: 0.3656 - val_acc: 0.8441
Epoch 5/10
56/56 [==============================] - 34s 614ms/step - loss: 0.3706 - acc: 0.8416 - val_loss: 0.3522 - val_acc: 0.8479
Epoch 6/10
56/56 [==============================] - 43s 772ms/step - loss: 0.3482 - acc: 0.8484 - val_loss: 0.2555 - val_acc: 0.8735
Epoch 7/10
56/56 [==============================] - 45s 801ms/step - loss: 0.3319 - acc: 0.8524 - val_loss: 0.1837 - val_acc: 0.9324
Epoch 8/10
56/56 [===

In [8]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

100/100 [==============================] - 3s 26ms/step


In [9]:
pred_bool = (pred >0.5)

In [10]:
predictions = pred_bool.astype(int)
columns=["desert", "mountains", "sea", "sunset", "trees"]
#columns should be the same order of y_col
results=pd.DataFrame(predictions, columns=columns)
results["Filenames"]=test_generator.filenames
ordered_cols=["Filenames"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv("results.csv",index=False)

In [12]:
predictions=[]
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)